In [20]:
#import necessary libraries
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [21]:
# initialize height, width and split data into train, validation
height = 299
width = 299
batch_size = 2

train_datagen = ImageDataGenerator(rescale=1./255,
    horizontal_flip=False,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'D:\.Downloads\high_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    'D:\.Downloads\high_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')


epochs = 30
steps = 170
val_steps = 40

Found 170 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [22]:
# create inception model and add 
base_model = InceptionV3(include_top=False, weights=None)#, classes=4)

new_model_in = base_model.get_layer(index=0).input
new_model_out = base_model.get_layer(index=-2).output
new_model = Model(inputs=new_model_in, outputs=new_model_out)

model = Sequential()
model.add(new_model)
model.add(Dense(4, activation='softmax', input_dim=2048))

LR = 0.001
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# own model NOT USED for project ----------------------------------------------------------------
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [23]:
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps // batch_size,
    validation_data=validation_generator,
    validation_steps=val_steps // batch_size
)

Epoch 1/30


ValueError: in user code:

    D:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    D:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(data)
    D:\anaconda3\lib\site-packag

In [15]:
accs = 0
for i in range(10):
    accuracy = model.evaluate(
        validation_generator,
        steps=val_steps // batch_size,
    )
    accs += accuracy[1]
print("avg accuracy: " + str(accs/10))

20/20 [==============================] - 2s 117ms/step - loss: 1.7299 - accuracy: 0.3250
avg accuracy: 0.32499998807907104


In [ ]:
model.save('D:\.Downloads\models\high_resv2')